In [ ]:
import pandas as pd 
import yfinance as yf
from pykrx import stock
from datetime import timedelta
import time
import warnings
# 경고를 무시하도록 설정
warnings.filterwarnings("ignore")
#stock.get_market_cap('20210501','20210510',"000480")
df= pd.read_csv('./data/자산,자본,부채,매출.csv',encoding='cp949')

In [ ]:
#인덱스가 티커인거 
#stock.get_market_cap("2019-04-01")
# 걍 이거 아래나 위나 다 일정시간 이상 접근하면 코드가 막히는 문제가 발생함
# https://github.com/sharebook-kr/pykrx/issues 확인할경우 걍 진짜 한국거래소에서 제한 걸어버린거임

In [ ]:
#인덱스 날짜인거 
#주의할점은 31일로 끝날짜 지정시 걍 값이 안불러와질 수 있음 
#stock.get_market_cap("2019-04-01", "2019-04-30", "058820")

In [ ]:
#stock.get_market_cap_by_date('20210501','20210510', "005930")
# 특정 거래소코드 시가총액 가져오기. 이게 안된다고 하셨음 

In [ ]:
import re

strs = ['(*)', '(IFRS연결)', '(천원)', '(IFRS)', '[제조]',
         '*','(-)',('결손금'),'(수익)','(보고서기재)','()','(정상)']

for s in strs:
    escape_s = re.escape(s)  # 문자열 이스케이프 처리 << 이거 안해줄경우 오류날 수 있음 import re 필수
    #이거 해줘야만 하는 이유는 ()가 특정 그룹처리가 되는데, 문자열로 읽으라고 명령하고 있는거임
    #이거 vscode는 안해줘도 되는데 주피터는 또 해줘야됨 아 ㅋㅋ
    print(escape_s)
    print(s)

### 복습용


- data폴더의 '자산,자본,부채,매출.csv'파일 df로 읽기
- 연결이 붙은 열을 추출해 개별 df로 만들기 (filtered_df) 그후 filtered_df에 동일한 이름의 df의 회사명, 거래소코드, 회계년도를 할당하기
- 기존의 df에서 열이름에 '연결'이 붙은 열을 드랍하기
- 두 데이터프레임의 열 이름을 깔끔하게 정리하기 (for문 사용, 제거할 내용을 리스트화하기)
- 기존 df와 filtered_df를 합치기. 이 때, filtered_df가 연결값이므로 우선시 되어야함 이름은 df_sum으로

### 문제1

- df_sum의 회계년도 열을 데이트타임으로 변경하기 
- 매수년도라는 열을 새롭게 만들고, 회계년도 에서 1년뒤 값을 넣기 (년도값만들어감)
- 년도값만 들어가 있는 매수년도 열을 str타입으로 바꾸고 '-04-01' 붙이기. 
- timedelta(import 필요)를 이용해 매수년도 값에 5일을 더해준 매수년도2열 만들기
- df_sum의 거래소코드를 제로패딩해주기 

### 문제2 df = df_sum.head()로 사용

- 거래소코드로 시가총액 불러오기
- 설명코드를 보고 for in zip을 사용해 시가총액 가져오기
- 시가총액 리스트를 만들고, 해당 리스트를 시가총액열에 넣어주기
- 현재 pykrx를 krx에서 차단해서 사용불가능 걍 코드 보여드리고 설명만해라 

In [ ]:
### 설명용
df=df_sum.head()

#stock.get_market_cap(df['매수년도'][0], df['매수년도2'][0], df['거래소코드'][0])

In [ ]:
### 설명용2
for i,j,c in zip(df['매수년도'],df['매수년도2'] ,df['거래소코드']):
    print(i, j, c)

In [ ]:
marcap=[]


for i,j,c in zip(df['매수년도'],df['매수년도2'] ,df['거래소코드']):
    marcap.append(stock.get_market_cap(i, j, c).head(1)['시가총액'][0])

df['시가총액'] =marcap
    

### pykrx는 현재 서버에서 차단중이므로 대안구하기

- 1. https://github.com/FinanceData/marcap <<깃허브 사용을 통한 시가총액 가져오기>>
- 2. http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101
- krx에서 일일이 가져온 뒤, merge나 concat써서 사용하기. 우선은 이거로 진행


### concat연습하기
- concat은 위아래로 붙이기

In [98]:
# 예시
df=pd.read_csv('./data/시가총액/2019.csv',encoding='cp949')
df['year'] = 2019
df2=pd.read_csv('./data/시가총액/2020.csv',encoding='cp949')
df2['year'] = 2020
df3=pd.concat([df, df2])
df3.sort_values('종목코드')

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,year
590,000020,동화약품,KOSPI,NaN,6580,30,0.46,6610,6780,6500,163650,1091441740,183789072600,27931470,2020
581,000020,동화약품,KOSPI,NaN,9420,20,0.21,9390,9480,9310,31132,292622960,263114447400,27931470,2019
104,000040,KR모터스,KOSPI,NaN,600,1,0.17,599,639,589,726893,441665347,28416610800,47361018,2020
105,000040,KR모터스,KOSPI,NaN,497,4,0.81,493,501,480,193986,95027511,94153705275,189444075,2019
250,000050,경방,KOSPI,NaN,8000,220,2.83,7780,8140,7780,11545,92303810,219322160000,27415270,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1921,950160,코오롱티슈진,KOSDAQ,외국기업(소속부없음),24150,-10300,-29.90,24150,24150,24150,253054,6112129600,1473573832500,61017550,2019
1994,950160,코오롱티슈진,KOSDAQ,외국기업(소속부없음),8010,0,0.00,0,0,0,0,0,489551575500,61117550,2020
71,950170,JTC,KOSDAQ,외국기업(소속부없음),4120,-135,-3.17,4275,4350,4100,208451,883236270,144222730040,35005517,2020
73,950170,JTC,KOSDAQ,외국기업(소속부없음),8900,0,0.00,8990,9280,8860,457756,4136724040,311549101300,35005517,2019


### 문제3 시가총액 무식하게 읽기
목적은 data폴더안의 시가총액폴더 안 파일을 모두 합치고, 연도를 구분해주는 것 
- for문을 이용
- 저장할 빈 df_marcap이라는 df만들기
- file에 파일명 할당하기
- df로 csv읽기
- df['year']에 해당년도값 할당하기
- df와 df_marcap을 concat하기 
- 거래소코드열을 만들고 종목코드 값 넣기
- 필요한 거래소코드, 시가총액, year 열만 남기고 df_marcap에 저장하기


# 문제 4
- df_marcap의 거래소코드, year을 기준으로 df_marcap정렬하기
- df_sum에서 회계년도열의 월 값이 12가 아닌 경우를 출력하기
- 월 값이 12가 아닌 거래소코드 데이터를 모두 버리고 df_12에 저장하기 (isin사용)
- df_12에 'year'열만들기. 매수년도열의 년도값과 동일한 값 넣기 
- 이후 merge를 통해 df_12와 df_marcap합치기. 이때 기준은 거래소코드, year 두 열로 하기 (final_df로 할당)
